## Chaining
We can **chain** transformations and action to create a computation **pipeline**
Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

### Start the `SparkContext`

In [0]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-3704061629036211> in <module>
      1 import numpy as np
      2 from pyspark import SparkContext
----> 3 sc = SparkContext(master="local[4]")

/databricks/spark/python/pyspark/context.py in __init__(self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls)
    143                 " is not allowed as it is a security risk.")
    144 
--> 145         SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
    146         try:
    147             self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

/databricks/spark/python/pyspark/context.py in _ensure_initialized(cls, instance, gateway, conf)
    356 
    357                     # Raise error if there is already a running Spark context
--> 358                     raise ValueError

In [0]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

7, 9, 5, 1, 0, 3, 3, 8, 8, 8, 3, 0, 9, 1, 8, 8, 2, 2, 6, 7, 6, 6, 6, 2, 2, 3, 8, 4, 9, 7, 7, 4, 2, 9, 1, 9, 6, 7, 6, 3, 9, 2, 5, 3, 0, 8, 1, 4, 9, 2, 2, 4, 7, 3, 2, 3, 1, 9, 6, 8, 9, 5, 3, 2, 6, 8, 5, 9, 9, 7, 0, 1, 9, 5, 1, 5, 4, 8, 3, 8, 3, 4, 0, 0, 4, 1, 5, 1, 0, 9, 1, 5, 3, 8, 4, 8, 0, 8, 4, 7, 6, 7, 6, 8, 9, 1, 4, 3, 7, 4, 9, 9, 9, 7, 4, 5, 0, 1, 5, 7, 5, 9, 2, 3, 1, 2, 7, 7, 5, 4, 9, 8, 2, 8, 2, 2, 7, 0, 0, 7, 8, 7, 5, 7, 8, 4, 7, 6, 1, 1, 8, 5, 7, 9, 0, 9, 4, 0, 8, 9, 9, 5, 1, 8, 7, 5, 1, 8, 7, 2, 0, 2, 3, 0, 4, 2, 8, 3, 7, 1, 0, 8, 8, 2, 2, 6, 8, 2, 3, 8, 0, 8, 9, 4, 1, 1, 4, 2, 9, 9, 4, 2, 8, 7, 8, 4, 0, 2, 3, 3, 1, 7, 6, 5, 2, 3, 3, 4, 1, 1, 9, 4, 7, 4, 3, 1, 3, 6, 7, 1, 2, 8, 8, 2, 9, 3, 1, 6, 4, 5, 7, 8, 6, 4, 3, 1, 0, 1, 8, 4, 6, 0, 9, 7, 3, 5, 7, 8, 6, 1, 4, 6, 0, 0, 4, 9, 3, 4, 0, 6, 6, 5, 0, 3, 0, 4, 6, 0, 8, 2, 5, 7, 5, 7, 2, 9, 2, 9, 2, 6, 4, 3, 4, 8, 6, 0, 6, 8, 7, 7, 5, 2, 8, 0, 8, 5, 4, 4, 7, 1, 8, 5, 7, 8, 3, 3, 4, 7, 2, 5, 3, 1, 9, 5, 9, 7, 9, 7, 9, 8, 7, 0, 5, 6

### Sequential syntax for chaining
Perform assignment after each computation

In [0]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

CPU times: user 10.8 ms, sys: 428 µs, total: 11.2 ms
Wall time: 880 ms


In [0]:
print(summation)

30412


### Cascaded syntax for chaining
Combine computations into a single cascaded command

In [0]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 8.18 ms, sys: 0 ns, total: 8.18 ms
Wall time: 299 ms
Out[8]: 30412

### Both syntaxes mean exactly the same thing
The only difference:
* In the sequential syntax the intermediate RDD has a name `Squares`
* In the cascaded syntax the intermediate RDD is *anonymous*

The execution is identical!

### Sequential execution
The standard way that the map and reduce are executed is
* perform the map
* store the resulting RDD in memory
* perform the reduce

### Disadvantages of Sequential execution

1. Intermediate result (`Squares`) requires memory space.
2. Two scans of memory (of `B`, then of `Squares`) - double the cache-misses.

### Pipelined execution
Perform the whole computation in a single pass. For each element of **`B`**
1. Compute the square
2. Enter the square as input to the `reduce` operation.

### Advantages of Pipelined execution

1. Less memory required - intermediate result is not stored.
2. Faster - only one pass through the Input RDD.

In [0]:
sc.stop()